In [1]:
import common as ak
from operator_plan import OperatorPlan
import numpy as np
import numpy.typing as npt

%load_ext autoreload
%autoreload 2

In [19]:
item_dict = ak.get_item_dict(lang="en_US")
#char_dict = ak.get_char_dict(lang="en_US")
char_dict = ak.get_char_dict(lang="zh_CN")

In [21]:
n_items = len(item_dict.keys())
n_chars = len(char_dict.keys())
item_names, item_names_rev = ak.get_item_info(item_dict)
#char_names, char_names_rev = ak.get_char_info(char_dict)
char_names, char_names_rev = ak.get_char_translations(char_dict)

In [22]:
char_ids, elite_costs, skill_costs, mastery_costs, module_costs = ak.get_all_char_all_costs(char_dict, n_chars)


In [23]:
## Testing
e_cost, s_cost, m_cost, _d_cost = ak.get_char_all_costs("Angelina", char_names_rev, char_ids,
                                      elite_costs, skill_costs, mastery_costs, module_costs)

ret = ak.sum_skill_slice(m_cost[0])
print(ret)
print(item_names[str(ret[6][0])])

[[ 3303    33]
 [30024     5]
 [30043     4]
 [30064     3]
 [30074     9]
 [30094     4]
 [30125     6]]
Bipolar Nanoflake


In [31]:
plan_nearl = OperatorPlan(
    name = "Nearl the Radiant Knight",
    elite_range = (0, 2),
    skill_range = (1, 7),
    mastery_range = [
        (0, 0),
        (0, 0),
        (0, 3)
    ]
)

e_cost, s_cost, m_cost, _d_cost = ak.get_char_all_costs(plan_nearl.name, char_names_rev, char_ids,
                                      elite_costs, skill_costs, mastery_costs, module_costs)

mats_combined = plan_nearl.get_total_cost(e_cost, s_cost, m_cost, _d_cost)

for i, c in mats_combined:
    print("{}: {}".format(item_names[str(i)], c))

Guard Chip: 5
Guard Dualchip: 4
Skill Summary - 1: 10
Skill Summary - 2: 24
Skill Summary - 3: 43
Orirock: 6
Orirock Concentration: 14
Sugar: 5
Polyester: 4
Oriron: 4
Oriron Cluster: 4
Polyketon: 4
Damaged Device: 4
Device: 5
Optimized Device: 3
Grindstone: 5
RMA70-12: 3
Polymerization Preparation: 4
Crystalline Electronic Unit: 6
Polymerized Gel: 12
Incandescent Alloy: 5


# Testing - Loading Gamepress from JSON

In [7]:
import json
with open("gamepress.json") as f:
    gpdata = json.load(f)["operators"]

In [8]:
n_gp_operators = len(gpdata)
gp_plans = np.empty(n_gp_operators, dtype=OperatorPlan)

In [9]:
for i, op in enumerate(gpdata):
    try:
        ## gamepress id doesnt match in-game id data
        name = char_names_rev[op["operator"]]
    except:
        name = op["operator"]
    print(name)
    gp_plans[i] = OperatorPlan(
            name = name,
            elite_range = (op["start_promotion"], op["end_promotion"]),
            skill_range = (op["start_skill"], op["end_skill"]),
            mastery_range = np.transpose((op["start_specs"], op["end_specs"]))
        )

9241
19141
13296
1241
21761
13281
1291
14701
16026
37136
37146
37176
35311
39001
25811
7211
23291
26601
30201
1296
19641
30156
1336
1376
16031
1306
24826
40436
40446
41791
26616
1311
11966
16036
1186
26536
42456
18066
43586
44496
44491
44501
14706
46546
42461
40451
43581


# Test - Serialising OperatorPlan objects

In [33]:
obj_str = plan_nearl.to_json()

new_dict = json.loads(obj_str)
new_plan = OperatorPlan(**new_dict)

print(new_plan)